# Getting started with Claude Opus 4.6 on Amazon Bedrock

**Build sophisticated AI applications with Anthropic's most powerful model**

This notebook demonstrates how to use Claude Opus 4.6 on Amazon Bedrock. As Anthropic's most intelligent model, Opus 4.6 excels at complex, multi-step projects requiring deep reasoning, long-horizon agentic tasks, and enterprise workflows demanding precision.

---

## What you'll learn

- **Basic usage**: Get started with Claude Opus 4.6 using the Converse and InvokeModel APIs
- **Adaptive thinking**: Leverage Claude's intelligent extended thinking mode
- **Context compaction**: Enable infinite context for long-running conversations (beta)

---

## When to use Claude Opus 4.6

Claude Opus 4.6 is ideal for:

- **Complex, multi-step projects** requiring deep reasoning
- **Long-horizon agentic tasks** with extended planning
- **Enterprise workflows** needing precision (finance, legal, compliance)
- **Computer use automation** for desktop and browser tasks
- **Advanced coding** — compresses multi-day projects into hours, handling full lifecycle from architecture to deployment

---

## Key capabilities

| Capability | Details |
|------------|--------|
| **Context window** | Up to 1M tokens (beta). Premium pricing applies for requests exceeding 200K tokens: 2x input / 1.5x output. |
| **Max output** | Up to 128K tokens |
| **Adaptive thinking** | Extended reasoning that activates when needed, with configurable effort levels |
| **Context compaction** | Automatic summarization for infinite context (beta, InvokeModel only) |

---

## 1. Setup and configuration

In [ ]:
# Install required packages (uncomment if needed)
# !pip install boto3 --upgrade

In [15]:
import boto3
import json
from botocore.config import Config

# Configuration - Verified working endpoint from discovery scan
REGION = "us-east-1"
MODEL_ID = "us.anthropic.claude-opus-4-6-v1"

# Config with longer timeout for thinking/compaction
FAST_CONFIG = Config(connect_timeout=5, read_timeout=120, retries={'max_attempts': 2})

# Initialize session with profile
session = boto3.Session(profile_name=PROFILE)

# Initialize the Bedrock Runtime client (production)
bedrock_runtime = session.client(
    service_name='bedrock-runtime',
    region_name=REGION,
    config=FAST_CONFIG
)

print(f"✓ Region: {REGION}")
print(f"✓ Model: {MODEL_ID}")
print(f"✓ Client initialized successfully")

✓ Region: us-east-1
✓ Model: us.anthropic.claude-opus-4-6-v1
✓ Client initialized successfully


---

## 2. Basic usage with Converse API

The Converse API provides a unified interface for conversational AI across Amazon Bedrock models.

*Note: Code examples are placeholders pending final API documentation.*

In [ ]:
# Basic request using the Converse API
response = bedrock_runtime.converse(
    modelId=MODEL_ID,
    messages=[
        {
            "role": "user",
            "content": [{"text": "What are three key considerations when designing a distributed system for high availability?"}]
        }
    ],
    inferenceConfig={
        "maxTokens": 2048,
        "temperature": 1
    }
)

print(response["output"]["message"]["content"][0]["text"])

---

## 3. Basic usage with InvokeModel API

The InvokeModel API provides direct access to Claude's native request format.

In [3]:
# Basic request using the InvokeModel API
request_body = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 2048,
    "temperature": 1,
    "messages": [
        {
            "role": "user",
            "content": [{"type": "text", "text": "Explain the CAP theorem and its implications for database design."}]
        }
    ]
}

response = bedrock_runtime.invoke_model(
    modelId=MODEL_ID,
    body=json.dumps(request_body)
)

response_body = json.loads(response["body"].read())
print(response_body["content"][0]["text"])

# CAP Theorem

## Core Concept

The CAP theorem, formulated by Eric Brewer in 2000 (and proven by Gilbert and Lynch in 2002), states that a distributed data system can only guarantee **two out of three** properties simultaneously:

```
        C
       / \
      /   \
     /     \
    A ───── P
```

## The Three Properties

### **C** — Consistency
Every read receives the **most recent write** or an error. All nodes see the same data at the same time.

```
Client writes X=5
  ↓
Node A: X=5  ──sync──  Node B: X=5
  ↓                      ↓
Both return X=5 (consistent)
```

### **A** — Availability
Every request receives a **non-error response**, without guarantee that it contains the most recent write. The system is always operational.

### **P** — Partition Tolerance
The system continues to operate despite **network failures** (messages lost or delayed between nodes). Nodes can't communicate but still serve requests.

```
Node A ──── ✗ network break ✗ ──── Node B
  ↓                    

---

## 4. Adaptive thinking

Adaptive thinking is an evolution of extended thinking that gives Claude the freedom to think if and when it determines reasoning is required. This optimizes both performance and speed.

### Two modes

| Mode | Description |
|------|-------------|
| **adaptive** | Claude decides whether to think based on query complexity |
| **enabled** | Thinking is guaranteed to start, with a specified token budget (model can exit early) |

With adaptive mode, Claude will think on complex queries but respond immediately for simple ones. With enabled mode and `budget_tokens`, thinking always begins but the model may conclude before reaching the budget.

In [8]:
# Enabled thinking with token budget

request_body = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 3000,
    "temperature": 1,
    "thinking": {
        "type": "enabled",
        "budget_tokens": 1024
    },
    "messages": [
        {
            "role": "user",
            "content": "Three players A, B, C play a game. Each has a jar with 100 balls numbered 1-100. Simultaneously, each draws one ball. A beats B if A's number > B's number (mod 100, treating 100 as 0 for comparison). Similarly for B vs C and C vs A. The overall winner is determined by majority of pairwise wins (ties broken randomly). Is there a mixed strategy Nash equilibrium where each player draws uniformly? If not, characterize the equilibrium."
        }
    ]
}

response = bedrock_runtime.invoke_model(
    modelId=MODEL_ID,
    body=json.dumps(request_body)
)

response_body = json.loads(response["body"].read())

# Display thinking and response
for block in response_body["content"]:
    if block["type"] == "thinking":
        print("🧠 Thinking:")
        print(block["thinking"][:500] + "...")
        print()
    elif block["type"] == "text":
        print("💬 Response:")
        print(block["text"])

🧠 Thinking:
Let me think carefully about this game.

Three players A, B, C each draw a ball numbered 1-100 from their respective jars. Let me denote their draws as $a, b, c$.

The pairwise comparisons: A beats B if $a > b$ (mod 100, treating 100 as 0). Wait, I need to understand the comparison rule more carefully.

"A beats B if A's number > B's number (mod 100, treating 100 as 0 for comparison)"

I think this means the comparison is done modulo 100, where 100 is treated as 0. So the numbers are effectively...



In [9]:
# Adaptive mode - Claude decides when to think

request_body = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 16000,
    "temperature": 1,
    "thinking": {
        "type": "adaptive"
    },
    "messages": [
        {
            "role": "user",
            "content": "Three players A, B, C play a game. Each has a jar with 100 balls numbered 1-100. Simultaneously, each draws one ball. A beats B if A's number > B's number (mod 100, treating 100 as 0 for comparison). Similarly for B vs C and C vs A. The overall winner is determined by majority of pairwise wins (ties broken randomly). Is there a mixed strategy Nash equilibrium where each player draws uniformly? If not, characterize the equilibrium."
        }
    ]
}

response = bedrock_runtime.invoke_model(
    modelId=MODEL_ID,
    body=json.dumps(request_body)
)

response_body = json.loads(response["body"].read())

# Check if thinking was triggered
has_thinking = any(block["type"] == "thinking" for block in response_body["content"])
print(f"Claude decided to think: {has_thinking}\n")

# Display content
for block in response_body["content"]:
    if block["type"] == "thinking":
        print("🧠 Thinking (first 300 chars):")
        print(block["thinking"][:300] + "...\n")
    elif block["type"] == "text":
        print("💬 Response:")
        print(block["text"][:800] + "..." if len(block["text"]) > 800 else block["text"])

Claude decided to think: True

💬 Response:



🧠 Thinking (first 300 chars):
Let me think carefully about this game.

Three players A, B, C each draw a ball from their jar (numbered 1-100). The pairwise comparisons determine the winner:

Wait, let me re-read the problem. "A beats B if A's number > B's number (mod 100, treating 100 as 0 for comparison)."

Hmm, this is a bit a...



In [10]:
# Simple query without thinking

request_body = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 2048,
    "temperature": 1,
    "messages": [
        {
            "role": "user",
            "content": "What is the capital of France?"
        }
    ]
}

response = bedrock_runtime.invoke_model(
    modelId=MODEL_ID,
    body=json.dumps(request_body)
)

response_body = json.loads(response["body"].read())
print(response_body["content"][0]["text"])

The capital of France is **Paris**.


---

## 5. Context compaction (beta)

Context compaction enables "infinite context" for long-running conversations and agentic tasks by automatically summarizing older context when approaching the context window limit.

**How it works:**
1. Enable compaction with `context_management.edits` containing a `compact_20260112` edit
2. When input tokens exceed a trigger threshold, Claude generates a summary
3. The API returns a compaction content block
4. Pass the compaction block back as part of the messages array on subsequent requests

**Important:**
- Context compaction is only available with the InvokeModel API (not Converse API)
- Requires `anthropic_beta: ["compact-2026-01-23"]` header
- Default trigger is ~500K tokens; customize with `trigger` parameter

In [12]:
# Basic compaction example

request_body = {
    "anthropic_version": "bedrock-2023-05-31",
    "anthropic_beta": ["compact-2026-01-12"],
    "max_tokens": 4096,
    "temperature": 1,
    "messages": [
        {
            "role": "user",
            "content": "Let's start a long-running conversation about building a complex e-commerce platform."
        }
    ],
    "context_management": {
        "edits": [
            {
                "type": "compact_20260112"
            }
        ]
    }
}

response = bedrock_runtime.invoke_model(
    modelId=MODEL_ID,
    body=json.dumps(request_body)
)

response_body = json.loads(response["body"].read())

# Check for compaction block
has_compaction = any(block.get("type") == "compaction" for block in response_body["content"])
print(f"Compaction triggered: {has_compaction}")

# Display response
for block in response_body["content"]:
    if block["type"] == "text":
        print("\n💬 Response:")
        print(block["text"][:500] + "..." if len(block["text"]) > 500 else block["text"])
    elif block.get("type") == "compaction":
        print("\n📦 Compaction block received - pass this back on subsequent requests")
        
print(f"\n📊 Usage: {response_body['usage']}")

Compaction triggered: False

💬 Response:
# E-Commerce Platform Design Conversation

Great, I'd love to dive into building a complex e-commerce platform with you! This is a rich topic that can span architecture, design patterns, databases, payments, scaling, and much more.

To get us started on the right track, let me ask a few foundational questions:

## Key Questions

1. **Scale & Scope**
   - What kind of products? (physical goods, digital, services, marketplace with multiple sellers?)
   - Expected scale? (hundreds of products vs. m...

📊 Usage: {'input_tokens': 95, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'cache_creation': {'ephemeral_5m_input_tokens': 0, 'ephemeral_1h_input_tokens': 0}, 'iterations': [{'input_tokens': 95, 'output_tokens': 405, 'cache_read_input_tokens': 0, 'cache_creation_input_tokens': 0, 'cache_creation': {'ephemeral_5m_input_tokens': 0, 'ephemeral_1h_input_tokens': 0}, 'type': 'message'}], 'output_tokens': 405}


In [14]:
# Multi-turn conversation with compaction and custom trigger

messages = []

def chat(user_message: str):
    """Send a message and handle compaction automatically"""
    messages.append({"role": "user", "content": user_message})
    
    request_body = {
        "anthropic_version": "bedrock-2023-05-31",
        "anthropic_beta": ["compact-2026-01-12"],
        "max_tokens": 4096,
        "messages": messages,
        "context_management": {
            "edits": [
                {
                    "type": "compact_20260112",
                    "trigger": {
                        "type": "input_tokens",
                        "value": 100000  # Trigger compaction at 100K tokens
                    }
                }
            ]
        }
    }
    
    response = bedrock_runtime.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps(request_body)
    )
    
    response_body = json.loads(response["body"].read())
    
    # Append assistant response (including any compaction blocks)
    messages.append({"role": "assistant", "content": response_body["content"]})
    
    # Display response
    for block in response_body["content"]:
        if block["type"] == "text":
            return block["text"]
    return ""

# Have a multi-turn conversation
print("Turn 1:")
print(chat("Design the database schema for an e-commerce platform"))
print("\n" + "="*50 + "\n")

print("Turn 2:")
print(chat("Now add the authentication and authorization system"))
print("\n" + "="*50 + "\n")

print("Turn 3:")
print(chat("What about payment processing integration?"))

Turn 1:
# E-Commerce Platform Database Schema

## Entity-Relationship Overview

```
┌──────────┐    ┌──────────────┐    ┌──────────┐    ┌──────────────┐
│  USERS   │───▶│  ADDRESSES   │    │ PRODUCTS │───▶│  CATEGORIES  │
└──────────┘    └──────────────┘    └──────────┘    └──────────────┘
     │                                   │  │
     │          ┌──────────────┐         │  │        ┌──────────────┐
     │─────────▶│   REVIEWS    │◀────────┘  │───────▶│  INVENTORY   │
     │          └──────────────┘            │        └──────────────┘
     │                                      │
     │          ┌──────────────┐            │        ┌──────────────┐
     │─────────▶│    CART      │────────────│───────▶│PRODUCT_IMAGES│
     │          └──────────────┘            │        └──────────────┘
     │                                      │
     │          ┌──────────────┐   ┌────────┴─────┐  ┌──────────────┐
     │─────────▶│   ORDERS     │──▶│ ORDER_ITEMS  │  │   COUPONS    │
     │     

---

## Summary

Claude Opus 4.6 is Anthropic's most powerful model, designed for demanding use cases where frontier intelligence matters most.

**Key features:**
- **Adaptive thinking** with effort levels (max, high, medium, low)
- **1M context window** (beta) with premium pricing for >200K tokens
- **Context compaction** (beta) for infinite context in agentic workflows
- **128K output tokens** for comprehensive responses

### Resources

- [Amazon Bedrock Documentation](https://docs.aws.amazon.com/bedrock/)
- [Anthropic Documentation](https://docs.anthropic.com/)